### WIP

In [2]:
import os
import numpy as np
import pandas as pd 
import pickle
import pymysql
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

### Cargamos nuevo df procesado con nuevas noticias

In [3]:
current_dir = os.path.abspath(os.getcwd())
print(f"Directorio actual: {current_dir}")

# Subir hasta el directorio raíz común (en este caso 'Analisis-de-noticias')
root_dir = os.path.abspath(os.path.join(current_dir, "../../.."))
print(f"Directorio raíz común: {root_dir}")

# Construir la ruta al directorio donde están los archivos pkl
directory = os.path.join(root_dir, "src/00.data/preprocesado/")
print(f"Ruta de directorio ajustada: {directory}")

# Lista para almacenar los DataFrames
df_lista = []

# Buscar todos los archivos pkl en el directorio
archivos_pkl = [f for f in os.listdir(directory) if f.startswith("meneame_procesado_") and f.endswith(".pkl")]

# Leer cada archivo .pkl y agregarlo a la lista de DataFrames
for archivo in archivos_pkl:
    file_path = os.path.join(directory, archivo)
    with open(file_path, "rb") as f:
        df_chunk = pickle.load(f)
        df_lista.append(df_chunk)
        print(f"Cargado: {archivo} con {len(df_chunk)} filas")

# Concatenar todos los DataFrames en uno solo
df = pd.concat(df_lista, ignore_index=True)

# Verificar el tamaño del DataFrame final
print(f"DataFrame final con {df.shape[0]} filas y {df.shape[1]} columnas")

Directorio actual: C:\Users\Jordi\OneDrive - Finergal\Documentos\Bootcamp HAB Data Science\PFB\Analisis-de-noticias\src\00.data\SQL
Directorio raíz común: C:\Users\Jordi\OneDrive - Finergal\Documentos\Bootcamp HAB Data Science\PFB\Analisis-de-noticias
Ruta de directorio ajustada: C:\Users\Jordi\OneDrive - Finergal\Documentos\Bootcamp HAB Data Science\PFB\Analisis-de-noticias\src/00.data/preprocesado/
Cargado: meneame_procesado_1.pkl con 100000 filas
Cargado: meneame_procesado_2.pkl con 100000 filas
Cargado: meneame_procesado_3.pkl con 87588 filas
DataFrame final con 287588 filas y 20 columnas


In [4]:
df.head(3)

,news_id,title,content,full_story_link,meneos,clicks,karma,positive_votes,anonymous_votes,negative_votes,category,comments,published_date,user,source,source_link,scraped_date,provincia,comunidad,category_encoded
0,4032594,RTVE mantiene que Mazón llegó al Cecopi antes ...,Los servicios informativos de la radio y la te...,https://meneame.net/story/rtve-mantiene-mazon-...,115,651,410,57,58,1,Política y Sociedad,30,2025-02-27 12:50:02,CosasQueLees,eldiario.es,https://www.eldiario.es/comunitat-valenciana/r...,2025-02-27 13:30:08,NaN,NaN,9.0
1,4032568,Arde Mississippi - ¿De donde sale todo ese odio?,Gene Hackman en la película Arde Mississippi (...,https://meneame.net/story/arde-mississippi-don...,77,982,362,50,27,2,Entretenimiento y Cultura,24,2025-02-27 12:35:02,Grahml,youtube.com,https://www.youtube.com/watch?v=dnWDqcdbANQ,2025-02-27 13:30:08,NaN,NaN,4.0
2,4032569,La Guardia Civil desaloja todos los colegios d...,La Guardia Civil está desalojando preventivame...,https://meneame.net/story/guardia-civil-desalo...,97,761,369,50,47,0,Política y Sociedad,70,2025-02-27 12:10:05,Tunguska08Chelyabinsk13,levante-emv.com,https://www.levante-emv.com/sucesos/2025/02/27...,2025-02-27 13:30:08,NaN,NaN,9.0


In [6]:
df = df.drop(columns=['full_story_link'], errors='ignore')

### Conectamos con SQL

In [7]:
# Cargar variables de entorno
load_dotenv()

user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("HOST", "localhost")
database = "meneame"

# Crear conexión con SQL
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

In [8]:
# Consultar los datos de las tablas en la base de datos antes de la importación
with engine.connect() as conn:
    # Consultas SQL para extraer los datos
    query_news_info = "SELECT * FROM news_info_table"
    query_user_info = "SELECT * FROM user_table"
    query_source_info = "SELECT * FROM source_table"
    
    # Leer los datos en DataFrame
    news_info_df = pd.read_sql(query_news_info, conn)
    user_info_df = pd.read_sql(query_user_info, conn)
    source_info_df = pd.read_sql(query_source_info, conn)

# Ver el tamaño de las tablas antes y después de la inserción
print(f"Total registros en news_info_table: {news_info_df.shape[0]}")
print(f"Total registros en user_table: {user_info_df.shape[0]}")
print(f"Total registros en source_table: {source_info_df.shape[0]}")

Total registros en news_info_table: 287563
Total registros en user_table: 25681
Total registros en source_table: 31088


In [9]:
# Extraer los datos existentes en la base de datos
with engine.connect() as conn:
    # Consultas SQL para extraer los IDs y valores correspondientes
    query_news_ids = "SELECT news_id FROM news_info_table"
    query_user_data = "SELECT user_id, user FROM user_table"
    query_source_data = "SELECT source_id, source FROM source_table"
    query_category_data = "SELECT category_id, category FROM category_table"
    query_location_data = "SELECT provincia_id, provincia, comunidad FROM location_table"
    
    # Extraer los datos
    news_ids_sql = pd.read_sql(query_news_ids, conn)
    user_data_sql = pd.read_sql(query_user_data, conn)
    source_data_sql = pd.read_sql(query_source_data, conn)
    category_data_sql = pd.read_sql(query_category_data, conn)
    location_data_sql = pd.read_sql(query_location_data, conn)

In [10]:
# Filtrar solo las noticias nuevas
new_news_df = df[~df['news_id'].isin(news_ids_sql['news_id'])]
new_news_df.shape

(25, 19)

In [11]:
# Obtener el último user_id y source_id
last_user_id = user_data_sql['user_id'].max()
last_source_id = source_data_sql['source_id'].max()

In [12]:
# Nuevos usuarios y fuentes
new_users = new_news_df[~new_news_df['user'].isin(user_data_sql['user'])]
new_sources = new_news_df[~new_news_df['source'].isin(source_data_sql['source'])]

# Insertar nuevos usuarios
if not new_users.empty:
    new_users_df = new_users[['user']].drop_duplicates()
    new_users_df['user_id'] = range(last_user_id + 1, last_user_id + 1 + len(new_users_df))
    new_users_df.to_sql('user_table', con=engine, if_exists='append', index=False)

# Insertar nuevas fuentes
if not new_sources.empty:
    new_sources_df = new_sources[['source']].drop_duplicates()
    new_sources_df['source_id'] = range(last_source_id + 1, last_source_id + 1 + len(new_sources_df))
    new_sources_df.to_sql('source_table', con=engine, if_exists='append', index=False)


In [13]:
# Nuevas categorías y ubicaciones
new_categories = new_news_df[~new_news_df['category'].isin(category_data_sql['category'])]
new_locations = new_news_df[~new_news_df['provincia'].isin(location_data_sql['provincia'])]

# Llenar con los ID correspondientes de forma segura
new_news_df.loc[:, 'category_id'] = new_news_df['category'].map(category_data_sql.set_index('category')['category_id'])
new_news_df.loc[:, 'provincia_id'] = new_news_df['provincia'].map(location_data_sql.set_index('provincia')['provincia_id'])

C:\Users\Jordi\AppData\Local\Temp\ipykernel_29660\1769496338.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_news_df.loc[:, 'category_id'] = new_news_df['category'].map(category_data_sql.set_index('category')['category_id'])
C:\Users\Jordi\AppData\Local\Temp\ipykernel_29660\1769496338.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_news_df.loc[:, 'provincia_id'] = new_news_df['provincia'].map(location_data_sql.set_index('provincia')['provincia_id'])


In [14]:
# Modificar de forma segura usando .loc
new_news_df.loc[:, 'user_id'] = new_news_df['user'].map(user_data_sql.set_index('user')['user_id'])
new_news_df.loc[:, 'source_id'] = new_news_df['source'].map(source_data_sql.set_index('source')['source_id'])

# Filtra las columnas necesarias para la inserción
new_news_df = new_news_df[['news_id', 'title', 'content', 'category_id', 'meneos', 'clicks', 'karma',
                           'positive_votes', 'anonymous_votes', 'negative_votes', 'comments', 'published_date',
                           'source_link', 'scraped_date', 'user_id', 'source_id', 'provincia_id']]

# Insertar las nuevas noticias en la base de datos
new_news_df.to_sql('news_info_table', con=engine, if_exists='append', index=False)

print("Proceso completado: Noticias, usuarios y fuentes actualizados.")

Proceso completado: Noticias, usuarios y fuentes actualizados.


C:\Users\Jordi\AppData\Local\Temp\ipykernel_29660\1794453794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_news_df.loc[:, 'user_id'] = new_news_df['user'].map(user_data_sql.set_index('user')['user_id'])
C:\Users\Jordi\AppData\Local\Temp\ipykernel_29660\1794453794.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_news_df.loc[:, 'source_id'] = new_news_df['source'].map(source_data_sql.set_index('source')['source_id'])


In [15]:
# Consultar los datos de las tablas en la base de datos
with engine.connect() as conn:
    # Consultas SQL para extraer los datos
    query_news_info = "SELECT * FROM news_info_table"
    query_user_info = "SELECT * FROM user_table"
    query_source_info = "SELECT * FROM source_table"
    
    # Leer los datos en DataFrame
    news_info_df = pd.read_sql(query_news_info, conn)
    user_info_df = pd.read_sql(query_user_info, conn)
    source_info_df = pd.read_sql(query_source_info, conn)

# Ver el tamaño de las tablas antes y después de la inserción
print(f"Total registros en news_info_table: {news_info_df.shape[0]}")
print(f"Total registros en user_table: {user_info_df.shape[0]}")
print(f"Total registros en source_table: {source_info_df.shape[0]}")

Total registros en news_info_table: 287588
Total registros en user_table: 25682
Total registros en source_table: 31089


In [ ]:
# al principio
# Total registros en news_info_table: 287563
# Total registros en user_table: 25681
# Total registros en source_table: 31088

### Cargamos df de Clustering para añadir info de Cluster

In [23]:
directorio_base = os.path.abspath(os.path.join(os.getcwd(), "../../.."))  # Subir dos directorios
directorio_pkl = os.path.join(directorio_base, "src", "00.data", "clustering")

# Obtener todos los archivos .pkl en el directorio de preprocesado
archivos_pkl = [f for f in os.listdir(directorio_pkl) if f.startswith("df_clustering_") and f.endswith(".pkl")]

# Si no se encuentran archivos .pkl, mostramos un mensaje
if not archivos_pkl:
    print("❌ No se encontraron archivos .pkl en el directorio.")
else:
    df_lista = []

    # Cargar todos los archivos .pkl
    for archivo in archivos_pkl:
        ruta_archivo = os.path.join(directorio_pkl, archivo)
        try:
            with open(ruta_archivo, "rb") as f:
                df_lista.append(pickle.load(f))
                print(f"✅ Archivo cargado correctamente: {archivo}")
        except FileNotFoundError:
            print(f"❌ No se encontró el archivo: {archivo}")
        except Exception as e:
            print(f"❌ Error al cargar {archivo}: {e}")

    # Concatenar todos los DataFrames en uno solo
    if df_lista:
        df_clustering = pd.concat(df_lista, ignore_index=True)
        print(f"✅ DataFrame final con {df_clustering.shape[0]} filas y {df_clustering.shape[1]} columnas")
    else:
        print("❌ No se cargaron DataFrames.")

✅ Archivo cargado correctamente: df_clustering_1.pkl
✅ Archivo cargado correctamente: df_clustering_2.pkl
✅ Archivo cargado correctamente: df_clustering_3.pkl
✅ DataFrame final con 287588 filas y 10 columnas


In [24]:
df_clustering.head(3)

,news_id,meneos,clicks,karma,positive_votes,negative_votes,anonymous_votes,comments,category,cluster
0,4032594,115,651,410,57,1,58,30,Política y Sociedad,0
1,4032568,77,982,362,50,2,27,24,Entretenimiento y Cultura,0
2,4032569,97,761,369,50,0,47,70,Política y Sociedad,0


In [32]:
df_clustering['cluster'].value_counts()

cluster
0    234443
1     44176
2      8969
Name: count, dtype: int64

In [25]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS cluster_table (
    cluster_id INT PRIMARY KEY,
    cluster_name VARCHAR(50)
);
"""

# Ejecutar la creación de la tabla
with engine.connect() as connection:
    connection.execute(text(create_table_sql))
    print("Tabla 'cluster_table' verificada o creada correctamente.")

Tabla 'cluster_table' verificada o creada correctamente.


In [26]:
# Definir los nombres de los clusters
cluster_names = {
    0: "Noticias polémicas",
    1: "Noticias estándar",
    2: "Noticias populares"
}

# Insertar los datos en la tabla
with engine.connect() as connection:
    for cluster_id, cluster_name in cluster_names.items():
        insert_sql = text("""
        INSERT INTO cluster_table (cluster_id, cluster_name)
        VALUES (:cluster_id, :cluster_name)
        ON DUPLICATE KEY UPDATE cluster_name = :cluster_name
        """)
        connection.execute(insert_sql, {"cluster_id": cluster_id, "cluster_name": cluster_name})
    print("Clusters insertados o actualizados correctamente.")

Clusters insertados o actualizados correctamente.


In [27]:
# Verificar que 'news_id' está en df_clustering
if "news_id" not in df_clustering.columns:
    raise KeyError("La columna 'news_id' no está en df_clustering. Verifica el DataFrame.")

# Crear diccionario {news_id: cluster_id}
news_cluster_mapping = df_clustering.set_index("news_id")["cluster"].to_dict()

# Actualizar los valores de cluster en la base de datos
update_sql = text("""
    UPDATE news_info_table
    SET cluster_id = :cluster_id
    WHERE news_id = :news_id
""")

with engine.connect() as connection:
    for news_id, cluster_id in news_cluster_mapping.items():
        connection.execute(update_sql, {"news_id": news_id, "cluster_id": cluster_id})

    print("Asignación de clusters a noticias completada.")

Asignación de clusters a noticias completada.


In [31]:
from sqlalchemy import text

update_sql = text("""
    UPDATE news_info_table
    SET cluster_id = :cluster_id
    WHERE news_id = :news_id;
""")

# Conexión a la base de datos
with engine.connect() as connection:
    for news_id, cluster_id in news_cluster_mapping.items():
        result = connection.execute(update_sql, {"news_id": news_id, "cluster_id": cluster_id})

print("Clusters actualizados en la base de datos.")

Clusters actualizados en la base de datos.


In [38]:
query = text("""
    SELECT c.cluster_name, COUNT(n.cluster_id) AS total_noticias
    FROM cluster_table c
    LEFT JOIN news_info_table n ON c.cluster_id = n.cluster_id
    GROUP BY c.cluster_name
    ORDER BY total_noticias DESC;
""")

with engine.connect() as connection:
    result = connection.execute(query)
    cluster_counts = result.fetchall()

for row in cluster_counts:
    print(row)

In [33]:
print(df_clustering.dtypes)

news_id             int64
meneos              int64
clicks              int64
karma               int64
positive_votes      int64
negative_votes      int64
anonymous_votes     int64
comments            int64
category           object
cluster             int32
dtype: object


In [35]:
# Obtener los news_id de la base de datos
news_ids_sql = pd.read_sql("SELECT news_id FROM news_info_table", engine)

# Comparar con df_clustering
common_news_ids = df_clustering[df_clustering["news_id"].isin(news_ids_sql["news_id"])]

print(f"Total de news_id en la BD: {len(news_ids_sql)}")
print(f"Total de news_id en df_clustering: {len(df_clustering)}")
print(f"Coincidencias de news_id: {len(common_news_ids)}")


Total de news_id en la BD: 287588
Total de news_id en df_clustering: 287588
Coincidencias de news_id: 287588


In [34]:
existing_news_ids = pd.read_sql("SELECT news_id FROM news_info_table", engine)
print(len(existing_news_ids), "news_id en la base de datos")
print(len(df_clustering), "news_id en df_clustering")
common_ids = df_clustering[df_clustering["news_id"].isin(existing_news_ids["news_id"])]
print(len(common_ids), "news_id coinciden y serán actualizados")

287588 news_id en la base de datos
287588 news_id en df_clustering
287588 news_id coinciden y serán actualizados


In [40]:
update_sql = text("""
    UPDATE news_info_table
    SET cluster_id = IFNULL(:cluster_id, 0)
    WHERE news_id = :news_id
""")

with engine.connect() as connection:
    transaction = connection.begin()
    rows_updated = 0
    for news_id, cluster_id in news_cluster_mapping.items():
        result = connection.execute(update_sql, {
            "news_id": int(news_id), 
            "cluster_id": int(cluster_id)  # Forzar tipo entero
        })
        rows_updated += result.rowcount
    transaction.commit()
    print(f"{rows_updated} filas actualizadas.")

287588 filas actualizadas.


In [37]:
print(df_clustering["news_id"].astype(str).str.len().unique())
print(pd.read_sql("SELECT LENGTH(news_id) FROM news_info_table LIMIT 10;", engine))

[7 6 5 4 3 1 2]
   LENGTH(news_id)
0                1
1                1
2                2
3                2
4                2
5                2
6                3
7                3
8                3
9                3


In [ ]:
######################

In [ ]:
IMPORTAR DF_CLUSTERING

In [ ]:
from sqlalchemy import text

# Crear la tabla de clusters si no existe
create_table_sql = """
CREATE TABLE IF NOT EXISTS cluster_table (
    cluster_id INT PRIMARY KEY,
    cluster_name VARCHAR(50)
);
"""
with engine.connect() as connection:
    connection.execute(text(create_table_sql))

# Insertar nombres de clusters en la tabla
cluster_names = {
    0: "Noticias polémicas",
    1: "Noticias estándar",
    2: "Noticias populares"
}

with engine.connect() as connection:
    for cluster_id, cluster_name in cluster_names.items():
        insert_sql = text("""
        INSERT INTO cluster_table (cluster_id, cluster_name)
        VALUES (:cluster_id, :cluster_name)
        ON DUPLICATE KEY UPDATE cluster_name = :cluster_name
        """)
        connection.execute(insert_sql, {"cluster_id": cluster_id, "cluster_name": cluster_name})

# Verificar que 'news_id' está en df_clustering
if "news_id" not in df_clustering.columns:
    raise KeyError("La columna 'news_id' no está en df_clustering.")

# Crear diccionario {news_id: cluster_id}
news_cluster_mapping = df_clustering.set_index("news_id")["cluster"].to_dict()

# Actualizar los valores de cluster en la base de datos
update_sql = text("""
    UPDATE news_info_table
    SET cluster_id = IFNULL(:cluster_id, 0)
    WHERE news_id = :news_id
""")

with engine.connect() as connection:
    transaction = connection.begin()
    for news_id, cluster_id in news_cluster_mapping.items():
        connection.execute(update_sql, {
            "news_id": int(news_id),
            "cluster_id": int(cluster_id)
        })
    transaction.commit()

print("Clusters actualizados en la base de datos.")
